In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import os


mohamedifqir_pixel_art_path = kagglehub.dataset_download('mohamedifqir/pixel-art')

print('Data source import complete.')

print(mohamedifqir_pixel_art_path)
print(os.listdir(mohamedifqir_pixel_art_path))


100%|██████████| 85.7M/85.7M [00:00<00:00, 117MB/s]

Extracting files...


Data source import complete.
/root/.cache/kagglehub/datasets/mohamedifqir/pixel-art/versions/1
['images', 'sprites.npy', 'sprites_labels.npy', 'labels.csv']


In [3]:
import shutil


# Assuming mohamedifqir_pixel_art_path is the original path
destination_path = '/content/pixel_art'  # New path in /content

# Move the directory
shutil.move(mohamedifqir_pixel_art_path, destination_path)

print(f"Data moved to: {destination_path}")
print(os.listdir(destination_path))

Data moved to: /content/pixel_art
['images', 'sprites.npy', 'sprites_labels.npy', 'labels.csv']


In [4]:

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d mohamedifqir/pixel-art -p /content
!unzip -q /content/pixel-art.zip -d /content/pixel_art

!ls /content/pixel_art


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
403 - Forbidden - Permission 'datasets.get' was denied
unzip:  cannot find or open /content/pixel-art.zip, /content/pixel-art.zip.zip or /content/pixel-art.zip.ZIP.
images	labels.csv  sprites_labels.npy	sprites.npy


In [4]:
!pip install mlflow
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.8 MB/s eta 0:00:00


In [5]:
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import mlflow
import wandb
from tqdm import tqdm
import numpy as np
from datetime import datetime
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image

In [6]:
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        x = F.relu(x)
        return x

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import mlflow
import mlflow.pytorch

class ContextResNet(nn.Module):
    def __init__(self, in_channels, n_feat=256, n_cfeat=10):
        super(ContextResNet, self).__init__()
        self.n_feat = n_feat
        self.n_cfeat = n_cfeat

        # Initial conv
        self.conv1 = nn.Conv2d(in_channels, n_feat, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(n_feat)

        # Downsampling layers
        self.downsample_layers = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(n_feat, n_feat, kernel_size=3, stride=2, padding=1),
                nn.BatchNorm2d(n_feat),
                nn.ReLU()
            ) for _ in range(2)
        ])

        self.layer1 = self._make_layer(n_feat, n_feat, 2)
        self.layer2 = self._make_layer(n_feat, n_feat * 2, 2)
        self.layer3 = self._make_layer(n_feat * 2, n_feat * 4, 2)

        self.time_embed = nn.Sequential(
            nn.Linear(1, n_feat * 4),
            nn.GELU(),
            nn.Linear(n_feat * 4, n_feat * 4)
        )

        self.context_embed = nn.Sequential(
            nn.Linear(n_cfeat, n_feat * 4),
            nn.GELU(),
            nn.Linear(n_feat * 4, n_feat * 4)
        )

        # Upsampling layers
        self.upsample_layers = nn.ModuleList([
            nn.Sequential(
                nn.ConvTranspose2d(
                    n_feat * 4 if i == 0 else n_feat,
                    n_feat,
                    kernel_size=4,
                    stride=2,
                    padding=1
                ),
                nn.BatchNorm2d(n_feat),
                nn.ReLU()
            ) for i in range(3)
        ])

        self.final = nn.Conv2d(n_feat, in_channels, 3, padding=1)

    def _make_layer(self, in_channels, out_channels, n_blocks):
        layers = []
        layers.append(ResNetBlock(in_channels, out_channels))
        for _ in range(1, n_blocks):
            layers.append(ResNetBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x, t, c=None):
        x = F.relu(self.bn1(self.conv1(x)))

        for layer in self.downsample_layers:
            x = layer(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        t_emb = self.time_embed(t.view(-1, 1)).view(-1, self.n_feat * 4, 1, 1)

        if c is None:
            c = torch.zeros(x.shape[0], self.n_cfeat).to(x.device)
        c_emb = self.context_embed(c).view(-1, self.n_feat * 4, 1, 1)

        x = x + t_emb + c_emb

        for layer in self.upsample_layers:
            x = layer(x)

        return self.final(x)

def perturb_input(x, t, noise, config):
    b_t = (config['beta2'] - config['beta1']) * torch.linspace(0, 1, config['timesteps'] + 1, device=x.device) + config['beta1']
    a_t = 1 - b_t
    ab_t = torch.cumsum(a_t.log(), dim=0).exp()
    ab_t[0] = 1

    t = t.long() - 1
    return ab_t.sqrt()[t, None, None, None] * x + (1 - ab_t[t, None, None, None]) * noise

# def train_model(model, dataloader, optimizer, device, config):
#     model.train()
#     mlflow.start_run()
#     for batch_idx, (x, _) in enumerate(dataloader):
#         x = x.to(device)
#         optimizer.zero_grad()

#         noise = torch.randn_like(x)
#         t = torch.randint(1, config['timesteps'] + 1, (x.shape[0],)).to(device)

#         x_pert = perturb_input(x, t, noise, config)
#         t_normalized = t.float() / config['timesteps']

#         pred_noise = model(x_pert, t_normalized)
#         loss = F.mse_loss(pred_noise, noise)
#         loss.backward()
#         optimizer.step()

#         if batch_idx % 10 == 0:
#             print(f"Batch {batch_idx}/{len(dataloader)}, Loss: {loss.item()}")
#             mlflow.log_metric("loss", loss.item(), step=batch_idx)

#     mlflow.pytorch.log_model(model, "model")
#     mlflow.end_run()

In [8]:
# Update the data transform to ensure proper image size
def get_transform(config):
    return transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((config['height'], config['height'])),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

# Update the CustomDataset to handle image resizing
class CustomDataset(Dataset):
    def __init__(self, sfilename, lfilename, transform, null_context=False):
        self.sprites = np.load(sfilename)
        self.slabels = np.load(lfilename)
        self.transform = transform
        self.null_context = null_context

    def __getitem__(self, idx):
        image = self.sprites[idx]
        if self.transform:
            image = self.transform(image)
        if self.null_context:
            label = torch.tensor(0).to(torch.int64)
        else:
            label = torch.tensor(self.slabels[idx]).to(torch.int64)
        return (image, label)

    def __len__(self):
        return len(self.sprites)

In [9]:
def train_model(model, dataloader, optimizer, device, config):
    wandb.init(project="diffusion-model", config=config)
    mlflow.start_run()
    mlflow.log_params(config)

    model.train()
    for epoch in range(config['n_epoch']):
        epoch_loss = 0.0
        pbar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{config["n_epoch"]}')

        for batch_idx, (x, _) in enumerate(pbar):
            x = x.to(device)
            optimizer.zero_grad()

            noise = torch.randn_like(x)
            t = torch.randint(1, config['timesteps'] + 1, (x.shape[0],)).to(device)

            x_pert = perturb_input(x, t, noise, config)
            pred_noise = model(x_pert, t / config['timesteps'])

            loss = F.mse_loss(pred_noise, noise)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            wandb.log({
                'batch_loss': loss.item(),
                'batch': batch_idx + epoch * len(dataloader)
            })

            pbar.set_postfix({'loss': loss.item()})

        avg_epoch_loss = epoch_loss / len(dataloader)

        mlflow.log_metric('epoch_loss', avg_epoch_loss, step=epoch)
        wandb.log({
            'epoch': epoch,
            'epoch_loss': avg_epoch_loss
        })

        if (epoch + 1) % config['save_interval'] == 0:
            checkpoint_path = os.path.join(config['save_dir'], f"model_epoch_{epoch+1}.pth")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_epoch_loss,
            }, checkpoint_path)
            mlflow.log_artifact(checkpoint_path)

    mlflow.end_run()
    wandb.finish()

In [10]:
def perturb_input(x, t, noise, config):
    b_t = (config['beta2'] - config['beta1']) * torch.linspace(0, 1, config['timesteps'] + 1, device=x.device) + config['beta1']
    a_t = 1 - b_t
    ab_t = torch.cumsum(a_t.log(), dim=0).exp()
    ab_t[0] = 1

    return ab_t.sqrt()[t, None, None, None] * x + (1 - ab_t[t, None, None, None]) * noise

In [11]:
def denoise_add_noise(x, t, pred_noise, z=None, config=None):
    if z is None:
        z = torch.randn_like(x)

    b_t = (config['beta2'] - config['beta1']) * torch.linspace(0, 1, config['timesteps'] + 1, device=x.device) + config['beta1']
    a_t = 1 - b_t
    ab_t = torch.cumsum(a_t.log(), dim=0).exp()

    noise = b_t.sqrt()[t] * z
    mean = (x - pred_noise * ((1 - a_t[t]) / (1 - ab_t[t]).sqrt())) / a_t[t].sqrt()
    return mean + noise

In [12]:
def sample_images(model, config, device, num_samples=16):
    model.eval()
    with torch.no_grad():
        samples = torch.randn(num_samples, 3, config['height'], config['height']).to(device)

        for i in range(config['timesteps'], 0, -1):
            print(f'sampling timestep {i:3d}', end='\r')
            t = torch.tensor([i / config['timesteps']])[:, None, None, None].to(device)
            z = torch.randn_like(samples) if i > 1 else 0

            eps = model(samples, t)
            samples = denoise_add_noise(samples, i, eps, z, config)

        return samples


In [18]:
def main():
    # Setup directories
    save_dir = './checkpoints'
    sample_dir = './samples'
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(sample_dir, exist_ok=True)

    # Configuration
    config = {
        'timesteps': 500,
        'beta1': 1e-4,
        'beta2': 0.02,
        # 'n_feat': 64,
        # 'n_cfeat': 5,
        'height': 16,  # Updated to match input image size
        'batch_size': 100,
        'n_epoch': 60,
        'learning_rate': 1e-3,
        'save_interval': 10,
        'save_dir': save_dir,
        'sample_dir': sample_dir
    }

    # Update dataset initialization
    transform = get_transform(config)
    dataset = CustomDataset(
        "/content/pixel_art/sprites.npy",
        "/content/pixel_art/sprites_labels.npy",
        transform,
        null_context=False
    )

    # Setup device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Setup data
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    dataset = CustomDataset(
        "/content/pixel_art/sprites.npy",
        "/content/pixel_art/sprites_labels.npy",
        transform,
        null_context=False
    )

    dataloader = DataLoader(
        dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    # Initialize model and optimizer
    model = ContextResNet(
        in_channels=3,
        # n_feat=config['n_feat'],
        # n_cfeat=config['n_cfeat']
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

    # Training
    try:
        train_model(model, dataloader, optimizer, device, config)

        # Generate samples
        samples = sample_images(model, config, device)
        save_image(samples, os.path.join(config['sample_dir'], 'final_samples.png'))

    except KeyboardInterrupt:
        print("\nTraining interrupted by user")
    except Exception as e:
        print(f"Error during training: {e}")
        raise e
    finally:
        # Save final model
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'config': config
        }, os.path.join(config['save_dir'], 'final_model.pth'))

if __name__ == "__main__":
    main()

Using device: cuda


Epoch 60/60: 100%|██████████| 894/894 [02:00<00:00,  7.44it/s, loss=0.42]


batch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇██
batch_loss,█▇▆▆▆▅▅▄▄▄▃▃▄▄▃▃▃▄▃▂▂▃▃▂▂▁▂▂▂▂▁▂▁▁▁▁▁▂▂▂
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇███
epoch_loss,██▆▆▅▄▄▄▄▃▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▂▂▂
batch,53639
batch_loss,0.42029
epoch,59
epoch_loss,0.46341


In [17]:
mlflow.end_run()

In [20]:
!pip install pyngrok --quiet

In [21]:
from pyngrok import ngrok

# Authenticate ngrok (optional, but recommended for better limits)
# Replace "YOUR_AUTHTOKEN" with your actual authtoken
# You can get an authtoken from the ngrok website after signing up
!ngrok authtoken "2k18JLz7XU8dn67xDCBKXcrS8aL_7psQCc4GmKqz2MrhumVLh"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [22]:
# Start the MLflow UI on a specific port (e.g., 5000)
get_ipython().system_raw("mlflow ui --port 5000 &")

# Get the public URL using ngrok
public_url = ngrok.connect(5000).public_url

print(f"MLflow UI is accessible at: {public_url}")

MLflow UI is accessible at: https://020a-34-125-247-149.ngrok-free.app
